# Chick-fil-A locations

In [1]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_grid as altgrid
from vega_datasets import data
import numpy as np
import circlify
import squarify
import matplotlib.pyplot as plt

/Users/stiles/.local/share/virtualenvs/notebooks-sxqKeO4C/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [3]:
%load_ext lab_black

---

## ZIP Codes

In [4]:
zips = pd.read_csv(
    "https://grid-news-static-files.s3.us-east-2.amazonaws.com/data/usa-zips-wealth.csv"
)

In [5]:
zips_pop = zips.sort_values("totpop_cy", ascending=False).head(1500)

---

## Locations

In [6]:
src = pd.read_csv("data/processed/locations.csv", dtype={"open_year": str})

In [7]:
df = src[src["status"] == "OPEN"].copy()

In [8]:
gdf = gpd.read_file("data/processed/locations_geo.geojson")

#### Decades

In [9]:
df["decade"] = df["open_year"].str[:3] + "0s"

---

## Aggregate

#### Operators

In [10]:
operators = (
    df.groupby(["operator"])
    .agg({"locationname": "count"})
    .reset_index()
    .rename(columns={"locationname": "count"})
    .sort_values("count", ascending=False)
)

#### Share of operators who own more than one? 

In [11]:
round((len(operators[operators["count"] > 1]) / len(operators)) * 100, 2)

16.67

#### Locations by year

In [12]:
df["open_year"] = df["open_year"].astype(str).str.replace(".0", "", regex=False)

In [13]:
years = (
    df.groupby(["open_year"])["url"]
    .count()
    .reset_index()
    .rename(columns={"url": "count", "open_year": "year"})
)

In [14]:
years.head()

,year,count
0,1946,1
1,1969,1
2,1970,2
3,1972,3
4,1973,2


In [15]:
years.tail()

,year,count
49,2018,97
50,2019,94
51,2020,89
52,2021,125
53,2022,52


#### States

In [16]:
states = (
    (
        df.groupby(["region"])["url"]
        .count()
        .reset_index()
        .rename(columns={"url": "count", "region": "state"})
    )
    .sort_values("count", ascending=False)
    .reset_index(drop=True)
)

In [17]:
states.head(8)

,state,count
0,TX,306
1,GA,161
2,FL,156
3,CA,137
4,NC,109
5,VA,86
6,OH,61
7,PA,57


#### Types over time

In [18]:
df.head()

,locationname,marketname,city,countrycode,address,zip,region,lat,lon,operator,status,timezone,url,open_day,open_month,open_year,other_date,playground,drivethru,format,in_mall,geometry,decade
0,Morton Ranch,"Houston, TX",Katy,US,2826 W Grand Pkwy N,77449,TX,29.814042,-95.772170,Amanda Baca,OPEN,America/Chicago,https://www.chick-fil-a.com/mortonranch,30.0,11.0,2017,2017-11-30,interior,True,Stand Alone,False,POINT (-95.7721704 29.8140416),2010s
1,Mason Road,"Houston, TX",Katy,US,369 S Mason Rd,77450,TX,29.780570,-95.751140,Rusty Wylie,OPEN,America/Chicago,https://www.chick-fil-a.com/masonroad,14.0,9.0,1995,2014-10-02,interior,True,Stand Alone,False,POINT (-95.75114 29.78057),1990s
2,Katy Green,"Houston, TX",Houston,US,19303 Katy Fwy,77094,TX,29.784326,-95.706519,Rusty Wylie,OPEN,America/Chicago,https://www.chick-fil-a.com/katygreen,11.0,6.0,2015,2015-06-11,interior,True,Stand Alone,False,POINT (-95.7065195 29.7843262),2010s
3,Katy Mills,"Houston, TX",Katy,US,25601 Nelson Way,77494,TX,29.774121,-95.819225,Cynthia Cavin,OPEN,America/Chicago,https://www.chick-fil-a.com/katymills,26.0,4.0,2012,2011-12-08,interior,True,Stand Alone,False,POINT (-95.819225 29.774121),2010s
4,Cinco Ranch,"Houston, TX",Katy,US,23860 Westheimer Pkwy,77494,TX,29.735510,-95.778000,Rusty Wylie,OPEN,America/Chicago,https://www.chick-fil-a.com/cincoranch,2.0,6.0,2005,2005-06-02,interior,True,Stand Alone,False,POINT (-95.778 29.73551),2000s


In [19]:
types_time = (
    df.groupby(["format", "open_year"])["url"]
    .count()
    .reset_index()
    .rename(columns={"url": "count", "open_year": "year"})
)

In [20]:
types_time.head()

,format,year,count
0,Food Court,1969,1
1,Food Court,1970,2
2,Food Court,1972,3
3,Food Court,1973,2
4,Food Court,1974,4


In [21]:
types_time["cumsum_type"] = types_time.groupby("format")["count"].cumsum()

In [22]:
types_time.head()

,format,year,count,cumsum_type
0,Food Court,1969,1,1
1,Food Court,1970,2,3
2,Food Court,1972,3,6
3,Food Court,1973,2,8
4,Food Court,1974,4,12


In [23]:
alt.Chart(types_time).mark_line().encode(
    x=alt.X("year:T", title="", axis=alt.Axis(tickCount=6)),
    y=alt.Y("cumsum_type", title="", axis=alt.Axis(tickCount=4)),
    color=alt.Color(
        "format",
        title="",
        scale=alt.Scale(
            domain=["Food Court", "In-Line", "Stand Alone"],
            range=["#4daf4a", "#377eb8", "#e41a1c"],
        ),
    ),
).configure_legend(
    orient="top", symbolType="stroke", offset=-30, padding=-35
).properties(
    width=650,
    height=300,
    title="Cumulative Chick-fil-A restaurant openings, by type",
)

alt.Chart(...)

In [24]:
df.format.value_counts()

Stand Alone    1536
Food Court      190
In-Line          67
Name: format, dtype: int64

---

In [25]:
df.groupby("format")["url"].count().reset_index()

,format,url
0,Food Court,190
1,In-Line,67
2,Stand Alone,1536


In [26]:
states = alt.topo_feature(data.us_10m.url, feature="states")

places = (
    alt.Chart()
    .mark_circle()
    .encode(
        longitude="lon:Q",
        latitude="lat:Q",
        size=alt.value(8),
        tooltip=["locationname", "city", "region"],
        color=alt.Color(
            "format",
            title=" ",
            scale=alt.Scale(
                domain=["Food Court", "In-Line", "Stand Alone"],
                range=["#4daf4a", "#377eb8", "#e41a1c"],
            ),
        ),
    )
    .project("albersUsa")
    .properties(width=250, height=150)
)

states = (
    alt.Chart(states).mark_geoshape(fill="#E9E9E9", stroke="white").project("albersUsa")
)

alt.layer(states, places, data=df).facet(
    facet=alt.Facet(
        "decade",
        title=" ",
        header=alt.Header(labelFontSize=14, labelFont="Summit Sans"),
    ),
    columns=4,
).configure_legend(orient="top", symbolType="circle").properties(
    title="Chick-fil-A openings, by location and decade"
)

alt.FacetChart(...)

In [27]:
df[df["city"] == "Houma"]

,locationname,marketname,city,countrycode,address,zip,region,lat,lon,operator,status,timezone,url,open_day,open_month,open_year,other_date,playground,drivethru,format,in_mall,geometry,decade
1715,Houma,"New Orleans, LA",Houma,US,1741 Martin Luther King Jr Blvd,70360,LA,29.61324,-90.75665,Shannon Lewis,OPEN,America/Chicago,https://www.chick-fil-a.com/houma,16.0,2.0,2006,2006-02-16,interior,True,Stand Alone,False,POINT (-90.75665 29.61324),2000s
